<a href="https://colab.research.google.com/github/AliBank/EfficientNetB3---Deep-Learning/blob/main/EfficientNetB3DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q keras
!pip install h5py pyyaml

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.optimizers import RMSprop
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.applications import EfficientNetB3
import matplotlib.pyplot as plt
import cv2
import numpy as np
from glob import glob
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/train/images/"
test_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/test/images/"

In [ ]:
numberOfClass=len(glob(train_path+"/*"))

In [ ]:
print(numberOfClass)

30


In [ ]:
eff = EfficientNetB3(include_top=False, weights='imagenet',input_shape=(224,224,3))
#transfer learning modeli yüklüyoruz

In [ ]:
print(eff.summary())

Model: "efficientnetb3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0

In [ ]:
eff.trainable=False
x =Flatten()(eff.output)
x =GlobalAveragePooling2D()(eff.output)
x= Dense(32, activation='relu')(x)
x= Dense(32, activation='relu')(x)
out=Dense(30,activation='softmax')(x)
model= Model(inputs=eff.inputs, outputs=out, name='EfficientNet_B3')
#yeni katmanlar ekliyoruz ve out katmanını 30 class olduğu için 30 yapıyoruz

print(len(model.layers))
for layer in model.layers:
  print(layer.name," ",layer.trainable)

for layer in model.layers[:10]:
  layer.trainable=False
  # katmanların yeniden ayarlanabilmesi için false yapıyoruz 
model.summary()  


In [ ]:
print(type(eff))

<class 'keras.engine.functional.Functional'>


In [ ]:
from keras.optimizers.optimizer_v2.rmsprop import RMSProp
optim_1 = RMSProp(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=optim_1, metrics=["accuracy"])
#modeli compile ediyoruz

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
#train
train_data= ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224))
test_data= ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224))
#fotolarımızı 224,224 formatına uyarlıyoruz.

Found 3601 images belonging to 30 classes.
Found 900 images belonging to 30 classes.


In [ ]:
batch_size=50

In [ ]:
#checkpointlerin kaydedileceği dosya ismini yazıyoruz. Dosya kendisi oluşuyor.
#sonra checkpointlerin parametlerini belirliyoruz. save_freq, steps_per_epoch * 5 olacak.
from keras.callbacks import ModelCheckpoint,CSVLogger
filepath="/content/drive/MyDrive/saved_models/weights-improvementEff-{epoch:05d}.ckpt"
checkpoint= ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False,save_freq=360, mode='max')
#log_csv =CSVLogger('my_logs.csv', separator=',', append=False)
callbacks_list =[checkpoint]

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=100,
                         validation_data= test_data,
                         validation_steps = 900//batch_size )

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=10,
                         initial_epoch=0,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementEff-00010.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=25,
                         initial_epoch=10,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementEff-00025.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=50,
                         initial_epoch=25,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementEff-00050.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=75,
                         initial_epoch=50,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementEff-00075.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=100,
                         initial_epoch=75,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementEff-00100.ckpt")

In [ ]:
model.save_weights("BaklagilFold1RMSProplr0.0001EfficientNetB3")